In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
df=pd.read_csv("train.csv")
df_original=pd.read_csv("train_dataset.csv")
df_test=pd.read_csv("test.csv")
df_test_original=pd.read_csv("train_dataset.csv")

In [3]:
X = df.drop(columns='smoking')
Y = df['smoking']
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=21)

In [4]:
#TRANSFORMACIÓN TRAIN
def outliers(x):
    x.loc[x["fasting blood sugar"]>350,"fasting blood sugar"] = x["fasting blood sugar"].mean()
    x.loc[x["triglyceride"]>700,"triglyceride"] = x["triglyceride"].mean()
    x.loc[x["LDL"]>1500,"LDL"] = x["LDL"].mean()
    x.loc[x["serum creatinine"]>350,"serum creatinine"] = x["serum creatinine"].mean()
    x.loc[x["AST"]>350,"AST"] = x["AST"].mean()
    x.loc[x["ALT"]>350,"ALT"] = x["ALT"].mean()
    x.loc[x["Gtp"]>350,"Gtp"] = x["Gtp"].mean()
    
def esc_log(x):
    variables = ['fasting blood sugar','triglyceride','LDL','serum creatinine','AST', 'ALT', 'Gtp']
    for i in variables:
        x.loc[:,i] = np.log(x[i])

def dummies(x):
    x = pd.get_dummies(x, drop_first=True, dtype=int)
    return x

def eliminar(x):
    x.drop(columns=['id','hearing(right)','hearing(left)','eyesight(right)','eyesight(left)'], inplace=True)
    return x
    
def transformacion(x):
    x["IMC"] = x["weight(kg)"]/((x["height(cm)"]/100)**2)
    x['eyesight'] = (x['eyesight(right)'] + x['eyesight(left)'])/2
    x['eyesightlvl'] = pd.cut(x.eyesight, bins=[0,0.56,1.11,5,10], labels=["a","b","c","d"])
    x['hearing'] = (x['hearing(right)'] + x['hearing(left)'])/2
    x["hearing"] = x["hearing"].replace(1.0,"no_sordo").replace(1.5,"medio_sordo").replace(2.0,"sordo")
    x['IMC_cat'] = pd.cut(x.IMC, bins=[0,18.5,24.9,29.9,46], labels=["underweight","normal_weight","overweight","obese"])
    x['pulse_pressure'] = x['systolic'] - x['relaxation']
    x['pressure'] = x['systolic'] + x['relaxation']
    x['pressure_cat'] = pd.cut(x.pressure, bins=[0,150,230,319], labels=['lowtension','normaltension','hypertension'])
    x['renal_funct'] = 186.3 * (x['serum creatinine'])**(-1.154) * (x['age'])**(-0.203) * (1/x['Urine protein'])
    df['age_cat'] = pd.cut(df.age, bins=[20,35,50,65,90], labels=['joven','adulto','mayor','anciano'])
    outliers(x)
    esc_log(x)
    x = dummies(x)
    eliminar(x)
    return x

In [5]:
x_train = transformacion(x_train)
#X = transformacion(X)

In [6]:
# ESCALAR Y MODELO XGBOOST
scale=StandardScaler() # paso 2
model=XGBClassifier(base_score=None, booster=None, callbacks=[],
              colsample_bylevel=0.28660569056196267, colsample_bynode=None,
              colsample_bytree=0.8276262525938465, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy='lossguide', importance_type=None,
              interaction_constraints=None, learning_rate=0.006892300208037651,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=0, max_leaves=38,
              min_child_weight=0.06601425602539084,
              monotone_constraints=None, multi_strategy=None,
              n_estimators=24969, n_jobs=-1, num_parallel_tree=None,
              random_state=None)
    
complete_proces=make_pipeline(scale, model) # todos los pasos juntos
#complete_proces.fit(X, Y)
complete_proces.fit(x_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=[],
                               colsample_bylevel=0.28660569056196267,
                               colsample_bynode=None,
                               colsample_bytree=0.8276262525938465, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None,
                               grow_policy='lossg...ype=None,
                               interaction_constraints=None,
                               learning_rate=0.006892300208037651, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=0, max_leaves=38,
                               min_child_weight=0.06601425602539084,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=24969,
                               n_jobs=-1, num_parallel_tree=None,
                               random_state=None, ...))])

In [7]:
#TRANSFORMACIÓN TEST
x_test = transformacion(x_test)

In [8]:
#PREDICCIÓN FINAL
probs = complete_proces.predict_proba(x_test)

In [ ]:
#VALIDACIÓN

In [9]:
from sklearn import metrics
print("Train AUC:", sklearn.metrics.roc_auc_score(y_test,probs[:,1]))

Train AUC: 0.8705169925483469


In [17]:
x_test=pd.read_csv("test.csv")

In [18]:
X_test = transformacion(x_test)

In [19]:
probs = complete_proces.predict_proba(X_test)

In [21]:
submission = pd.DataFrame({"id":x_test["id"], "smoking":probs[:,1]})
submission

,id,smoking
0,159256,0.646276
1,159257,0.204701
2,159258,0.517683
3,159259,0.015772
4,159260,0.645875
...,...,...
106166,265422,0.554255
106167,265423,0.513695
106168,265424,0.478148
106169,265425,0.074099


In [23]:
submission.to_csv("sample_submission.csv", encoding='utf-8', index=False) #Guardar dataframe final